In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
scaler = StandardScaler()
import pandas_profiling
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score as geo
from imblearn.metrics import make_index_balanced_accuracy as iba
from sklearn.metrics import roc_curve, auc
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from pycaret.regression import *
import sqlite3
from sqlite3 import Error
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier     
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix,classification_report       

In [2]:
df=pd.read_csv("profiles.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60552 entries, 0 to 60551
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          9514 non-null   float64
 1   body_type    8666 non-null   object 
 2   diet         5761 non-null   object 
 3   drinks       9012 non-null   object 
 4   drugs        7215 non-null   object 
 5   education    8459 non-null   object 
 6   essay0       8667 non-null   object 
 7   essay1       8347 non-null   object 
 8   essay2       8049 non-null   object 
 9   essay3       7690 non-null   object 
 10  essay4       7879 non-null   object 
 11  essay5       7814 non-null   object 
 12  essay6       7352 non-null   object 
 13  essay7       7553 non-null   object 
 14  essay8       6361 non-null   object 
 15  essay9       7554 non-null   object 
 16  ethnicity    8565 non-null   object 
 17  height       9514 non-null   float64
 18  income       9514 non-null   float64
 19  job 

In [4]:
pandas_profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df=df.drop_duplicates()[:9514]

In [6]:
df.drop(["ethnicity","job"],axis=1,inplace=True)

In [7]:
for i in df:
    if ("essay" in i):
        df.drop(i,axis=1,inplace=True)

In [8]:
skinnystatus={"average":"fit",
              "fit":"fit",
              "athletic":"fit",
              "thin":"fit",
              "curvy":"curvy",
              "a little extra":"curvy",
              "skinny":"fit",
              "full figured":"curvy",
              "overweight":"bad",
              "jacked":"bad",
              "used up":"bad",
              "rather not say":"curvy"
             }
df.body_type.replace(skinnystatus,inplace=True)

In [9]:
df["religion"]=df["religion"].str.split().str[0]

In [10]:
status={"mostly":"","strictly":""}
df["diet"].replace(status,regex=True,inplace=True)
df["diet"]=df["diet"].str.strip()

In [11]:
df["education"].replace({"working on":"student","graduated from":"graduated","dropped out":"dropped"},regex=True)
df["education"]=df["education"].str.split().str[0]
df["education"].replace({"college/university":"student","two-year":"student","masters":"student","working":"student","space":"student","ph.d":"graduated","law":"graduated","high":"graduated"},regex=True,inplace=True)

In [12]:
df["last_online_year"]=2021-(df['last_online'].str[:4]).astype(int)
df["last_online_month"]=(df['last_online'].str[5:7]).astype(int)
df.drop('last_online',axis=1)

,age,body_type,diet,drinks,drugs,education,height,income,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status,last_online_year,last_online_month
0,22.0,curvy,anything,socially,never,student,75.0,-1.0,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism,m,gemini,sometimes,english,single,9,6
1,35.0,fit,other,often,sometimes,student,70.0,80000.0,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism,m,cancer,no,"english (fluently), spanish (poorly), french (...",single,9,6
2,38.0,fit,anything,socially,NaN,graduated,68.0,-1.0,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available,9,6
3,23.0,fit,vegetarian,socially,NaN,student,71.0,20000.0,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single,9,6
4,29.0,fit,NaN,socially,never,graduated,66.0,-1.0,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9509,28.0,fit,anything,socially,never,student,73.0,60000.0,"san francisco, california","doesn&rsquo;t have kids, but wants them",straight,likes dogs and likes cats,christianity,m,aries and it&rsquo;s fun to think about,no,"english, spanish (okay)",single,9,6
9510,31.0,fit,anything,socially,sometimes,NaN,72.0,-1.0,"san francisco, california",NaN,gay,NaN,atheism,m,scorpio but it doesn&rsquo;t matter,sometimes,english,single,9,4
9511,34.0,fit,NaN,socially,NaN,dropped,71.0,-1.0,"san francisco, california",doesn&rsquo;t want kids,gay,likes dogs and has cats,atheism,m,leo and it&rsquo;s fun to think about,when drinking,english (fluently),single,9,4
9512,29.0,fit,anything,rarely,never,student,70.0,-1.0,"san francisco, california","doesn&rsquo;t have kids, and doesn&rsquo;t wan...",gay,likes dogs and has cats,NaN,m,aries,no,"english (fluently), french (okay), swedish (po...",single,9,3


In [13]:
df.location=df.location.str.split(",").str[0]
plt.figure(figsize=(15,5))

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

In [14]:
offsprings=[]
for i in df.offspring.fillna("bilinmiyor").str.replace("[^\w\s]",""):
    if "doesnrsquot" in i:
        offsprings.append("0")
    elif "bilinmiyor" in i:
        offsprings.append("0")
    else:
        offsprings.append("1")
        
df["offspring"]=offsprings
df["offspring"]=df["offspring"].astype(int)

In [15]:
df["sign"]=df["sign"].str.split().str[0]

In [16]:
df["number_of_fluent_languages"]=df.speaks.str.count("fluently")
df["number_of_fluent_languages"].replace(0,1,regex=True,inplace=True)
df["number_of_poorly_languages"]=df.speaks.str.count("poorly")
df.drop("speaks",axis=1,inplace=True)

In [17]:
for column in df.select_dtypes(include=np.number):
    df[column].fillna(df[column].median(),inplace=True)

In [18]:
for column in df.select_dtypes(include=np.object):
    df[column].fillna(df[column].mode()[0],inplace=True)

In [19]:
cag=[]
for i in df["age"].values:
    if (i>=18) and (i<=32):
        cag.append("Millennial")
    elif (i>=33) and (i<=47):
        cag.append("Gen X-er")
    elif (48>=33) and (i<=70):
        cag.append("Boomers")
    else:
        cag.append("Unkown")

df["cag"]=cag

In [20]:
df["cag"].replace("Unkown","Boomers",regex=True,inplace=True)

In [32]:
df.cag=df.cag.str.replace('Millennial','0')
df.cag=df.cag.str.replace('Gen X-er','1')
df.cag=df.cag.str.replace('Boomers','2')

In [33]:
df.cag=df.cag.astype(int)

In [21]:
df.corr()

,age,height,income,offspring,last_online_year,last_online_month,number_of_fluent_languages,number_of_poorly_languages
age,1.000000,-0.022078,0.011424,0.341753,-0.015792,-0.003527,-0.024795,-0.001559
height,-0.022078,1.000000,0.073105,-0.033389,-0.016633,-0.010858,0.015458,0.036414
income,0.011424,0.073105,1.000000,0.015045,0.009376,0.002043,0.039186,0.028577
offspring,0.341753,-0.033389,0.015045,1.000000,0.029129,0.005697,0.000841,-0.012303
last_online_year,-0.015792,-0.016633,0.009376,0.029129,1.000000,0.685183,-0.003813,-0.034827
last_online_month,-0.003527,-0.010858,0.002043,0.005697,0.685183,1.000000,-0.006593,-0.006141
number_of_fluent_languages,-0.024795,0.015458,0.039186,0.000841,-0.003813,-0.006593,1.000000,0.067774
number_of_poorly_languages,-0.001559,0.036414,0.028577,-0.012303,-0.034827,-0.006141,0.067774,1.000000


In [22]:
df_reg=pd.get_dummies(df,drop_first=True)

In [23]:
x,y=df_reg.drop("age",axis=1),df[["age"]]

In [24]:
def regression_funct(x,y): 
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

    L=LinearRegression()
    R=Ridge()
    Lass=Lasso()
    E=ElasticNet()
    ExTree=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    KN=KNeighborsRegressor()
    
    algos=[L,R,Lass,E,ExTree,GBR,KN]
    algo_names=['LinearRegression','Ridge','Lasso','ElasticNet','ExtraTreeRegressor','GradientBoostingRegressor','KNeighborsRegressor']
    r_squared=[]
    rmse=[]
    mae=[]
    
    result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
    
    for item in algos:
        item.fit(x_train,y_train)
        item.predict(x_test)
        r_squared.append(r2_score(y_test,item.predict(x_test)))
        rmse.append((mean_squared_error(y_test,item.predict(x_test)))**.5)
        mae.append(mean_absolute_error(y_test,item.predict(x_test)))
        
    result.R_Squared=r_squared
    result.RMSE=rmse
    result.MAE=mae
        
    return result.sort_values('R_Squared',ascending=False)

In [25]:
regression_funct(x,y)

,R_Squared,RMSE,MAE
GradientBoostingRegressor,0.853266,3.651839,2.974312
Ridge,0.837541,3.842541,3.093045
LinearRegression,0.804106,4.219453,3.359626
ExtraTreeRegressor,0.735889,4.899355,3.793484
Lasso,0.572190,6.235505,4.213471
KNeighborsRegressor,0.448128,7.082153,4.724120
ElasticNet,0.308607,7.926994,5.422564


In [35]:
def classification_funct(x,y):
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    
    g=GaussianNB()
    b=BernoulliNB()
    KN=KNeighborsClassifier()
    D=DecisionTreeClassifier()
    R=RandomForestClassifier()
    Log=LogisticRegression()
    XGB=XGBClassifier()
    
    algos=[g,b,KN,D,R,Log,XGB]
    algo_names=['GaussianNB','BernoulliNB','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','LogisticRegression','XGBClassifier']
   
    accuracy_scored=[]

    
    for item in algos:
        item.fit(x_train,y_train)
        item.predict(x_test)
        
        accuracy_scored.append(accuracy_score(y_test,item.predict(x_test)))
       
    result=pd.DataFrame(columns=['accuracy_score'],index=algo_names)
    result['accuracy_score']=accuracy_scored
    return result.sort_values('accuracy_score',ascending=False)

df_class=pd.get_dummies(df,drop_first=True)
x,y=df_class.drop(['cag','age'],axis=1),df[['cag']]

In [36]:
y=df[["cag"]]

In [37]:
df_class=df.drop(["cag","age"],axis=1)
x=pd.get_dummies(df_class,drop_first=True)

In [38]:
classification_funct(x,y)

,accuracy_score
XGBClassifier,0.676301
RandomForestClassifier,0.671046
BernoulliNB,0.662112
GaussianNB,0.642144
LogisticRegression,0.642144
KNeighborsClassifier,0.622701
DecisionTreeClassifier,0.620074
